# Prep lib and dataset

In [1]:
# make sure this ipnb file is in the same folder as Fine_tuning_a_masked_language_model...ipynb
import pandas as pd
import csv
import math
import os
import re

In [2]:
os.getcwd()

'/Users/yancong/Desktop/5 zili research/naacl2022_cong_shortPaper'

In [3]:
from transformers import pipeline

In [7]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

[{'score': 0.05292867496609688,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i'm a role model."},
 {'score': 0.039685964584350586,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i'm a fashion model."},
 {'score': 0.034743648022413254,
  'token': 2449,
  'token_str': 'business',
  'sequence': "hello i'm a business model."},
 {'score': 0.034623000770807266,
  'token': 2944,
  'token_str': 'model',
  'sequence': "hello i'm a model model."},
 {'score': 0.018145160749554634,
  'token': 11643,
  'token_str': 'modeling',
  'sequence': "hello i'm a modeling model."}]

In [2]:
unmasker("Hello I'm a [MASK] model.")[0]['score']

0.0529286190867424

# Distill-BERT

## quantifier

### dataset

In [5]:
quantifier_df = pd.read_csv('/Users/yancong/Desktop/5 zili research/cogsci2022/data/SI.csv', index_col=0)
quantifier_df.head()

,pre_cw,cw,post_cw,condition,prediction
0,Some people have,lungs,that are diseased by viruses .,baseline,so-so
1,Some people have,pets,that are diseased by viruses .,baseline,good
2,Some people have,lungs,which require good care .,experimental,bad
3,Some people have,pets,which require good care .,experimental,good
4,Some houses have,rooms,that are filled with kid's toys .,baseline,so-so


### playground

In [6]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
unmasker("[MASK] people have lungs that are diseased by viruses.")

[{'score': 0.20906086266040802,
  'token': 2070,
  'token_str': 'some',
  'sequence': 'some people have lungs that are diseased by viruses.'},
 {'score': 0.12402084469795227,
  'token': 2116,
  'token_str': 'many',
  'sequence': 'many people have lungs that are diseased by viruses.'},
 {'score': 0.11152411997318268,
  'token': 2087,
  'token_str': 'most',
  'sequence': 'most people have lungs that are diseased by viruses.'},
 {'score': 0.027400676161050797,
  'token': 2122,
  'token_str': 'these',
  'sequence': 'these people have lungs that are diseased by viruses.'},
 {'score': 0.022801494225859642,
  'token': 9750,
  'token_str': 'elderly',
  'sequence': 'elderly people have lungs that are diseased by viruses.'}]

In [11]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
unmasker("Some people have [MASK] that are diseased by viruses.")

[{'score': 0.09371289610862732,
  'token': 11595,
  'token_str': 'organs',
  'sequence': 'some people have organs that are diseased by viruses.'},
 {'score': 0.06411033868789673,
  'token': 21434,
  'token_str': 'tumors',
  'sequence': 'some people have tumors that are diseased by viruses.'},
 {'score': 0.036154840141534805,
  'token': 15245,
  'token_str': 'infections',
  'sequence': 'some people have infections that are diseased by viruses.'},
 {'score': 0.03473655879497528,
  'token': 7870,
  'token_str': 'diseases',
  'sequence': 'some people have diseases that are diseased by viruses.'},
 {'score': 0.03452983498573303,
  'token': 25409,
  'token_str': 'cancers',
  'sequence': 'some people have cancers that are diseased by viruses.'}]

In [14]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
unmasker("[MASK] people have lungs which require good care.")

[{'score': 0.14691618084907532,
  'token': 2070,
  'token_str': 'some',
  'sequence': 'some people have lungs which require good care.'},
 {'score': 0.09397731721401215,
  'token': 2116,
  'token_str': 'many',
  'sequence': 'many people have lungs which require good care.'},
 {'score': 0.09147515147924423,
  'token': 9750,
  'token_str': 'elderly',
  'sequence': 'elderly people have lungs which require good care.'},
 {'score': 0.08165210485458374,
  'token': 2087,
  'token_str': 'most',
  'sequence': 'most people have lungs which require good care.'},
 {'score': 0.04177016019821167,
  'token': 12419,
  'token_str': 'deaf',
  'sequence': 'deaf people have lungs which require good care.'}]

### dataset cw quantifier

In [ ]:
#https://support.microsoft.com/en-us/office/combine-text-from-two-or-more-cells-into-one-cell-81ba0946-ce78-42ed-b3c3-21340eb164a6
# then copy paste that column [value only]

In [45]:
new_qdf = pd.read_csv('/Users/yancong/Desktop/5 zili research/cogsci2022/data/SI_cw_quantifier.csv', index_col=0)
new_qdf.head()

,cw,post_cw,condition,prediction,sentence
0,some,people have lungs that are diseased by viruses.,baseline,so-so,Some people have lungs that are diseased by vi...
1,some,people have pets that are diseased by viruses.,baseline,good,Some people have pets that are diseased by vir...
2,all,people have lungs which require good care.,experimental,bad,Some people have lungs which require good care.
3,some,people have pets which require good care.,experimental,good,Some people have pets which require good care.
4,some,houses have rooms that are filled with kid's t...,baseline,so-so,Some houses have rooms that are filled with ki...


In [46]:
new_qdf['distillbert_cw_prob'] = ''
new_qdf['distillbert_top1'] = ''
new_qdf['distillbert_top1_prob'] = ''
new_qdf['distillbert_cw_accuracy'] = 0

unmasker = pipeline('fill-mask', model='distilbert-base-uncased')

for i in new_qdf.index:
    seq = "[MASK] " + new_qdf['post_cw'][i]
    temp = unmasker(seq)
    new_qdf['distillbert_top1'][i] = temp[0]['token_str']
    new_qdf['distillbert_top1_prob'][i] = temp[0]['score']

    for item in temp:
        if item['token_str'] == new_qdf['cw'][i]:
            new_qdf['distillbert_cw_prob'][i] = item['score']
            new_qdf['distillbert_cw_accuracy'][i] = 1


/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_5787/3803522104.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_qdf['distillbert_top1'][i] = temp[0]['token_str']
/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_5787/3803522104.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_qdf['distillbert_top1_prob'][i] = temp[0]['score']
/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_5787/3803522104.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [47]:
new_qdf.head()

,cw,post_cw,condition,prediction,sentence,distillbert_cw_prob,distillbert_top1,distillbert_top1_prob,distillbert_cw_accuracy
0,some,people have lungs that are diseased by viruses.,baseline,so-so,Some people have lungs that are diseased by vi...,0.209061,some,0.209061,1
1,some,people have pets that are diseased by viruses.,baseline,good,Some people have pets that are diseased by vir...,0.289194,some,0.289194,1
2,all,people have lungs which require good care.,experimental,bad,Some people have lungs which require good care.,,some,0.146916,0
3,some,people have pets which require good care.,experimental,good,Some people have pets which require good care.,0.228662,some,0.228662,1
4,some,houses have rooms that are filled with kid's t...,baseline,so-so,Some houses have rooms that are filled with ki...,0.037414,these,0.082235,1


## presupposition

In [4]:
df = pd.read_csv('/Users/yancong/Desktop/5 zili research/cogsci2022/data/Presp.csv', index_col=0)
df.head()

,pre_cw,cw,post_cw,post_cw_trigger,condition,prediction
0,Seth went to,jail,on Saturday night.,The guard spoke to him there for a while .,experimental,good
1,Seth went to a,restaurant,on Saturday night.,The guard spoke to him there for a while .,baseline,bad
2,Kristen went to a,restaurant,in the morning.,The waiter served her there quickly .,experimental,good
3,Kristen went to,jail,in the morning.,The waiter served her there quickly .,baseline,bad
4,Sue went to a,salon,in the fall.,The hairdresser addressed her there politely .,experimental,good


In [5]:
df['distillbert_cw_prob'] = ''
df['distillbert_top1'] = ''
df['distillbert_top1_prob'] = ''
df['distillbert_cw_accuracy'] = 0

unmasker = pipeline('fill-mask', model='distilbert-base-uncased')

for i in df.index:
    seq = df['pre_cw'][i] + " [MASK] " + df['post_cw'][i] + ' ' + df['post_cw_trigger'][i]
    temp = unmasker(seq)
    df['distillbert_top1'][i] = temp[0]['token_str']
    df['distillbert_top1_prob'][i] = temp[0]['score']

    for item in temp:
        if item['token_str'] == df['cw'][i]:
            df['distillbert_cw_prob'][i] = item['score']
            df['distillbert_cw_accuracy'][i] = 1

<ipython-input-5-12f6037b417f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distillbert_top1'][i] = temp[0]['token_str']
<ipython-input-5-12f6037b417f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distillbert_top1_prob'][i] = temp[0]['score']
<ipython-input-5-12f6037b417f>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distillbert_cw_prob'][i] = item['score']
<ipython-input-5-12f6037b417f>:17: SettingWithCopyWa

# Fine-tuned distill-bert

## quantifier

### playground

In [ ]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased-finetuned-quantifier')
unmasker("[MASK] people have lungs that are diseased by viruses.")

In [12]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased-finetuned-quantifier')
unmasker("Some people have [MASK] that are diseased by viruses.")

[{'score': 0.08351080864667892,
  'token': 8860,
  'token_str': 'libraries',
  'sequence': 'some people have libraries that are diseased by viruses.'},
 {'score': 0.05673876032233238,
  'token': 9941,
  'token_str': 'museums',
  'sequence': 'some people have museums that are diseased by viruses.'},
 {'score': 0.05336903780698776,
  'token': 3765,
  'token_str': 'cars',
  'sequence': 'some people have cars that are diseased by viruses.'},
 {'score': 0.042930975556373596,
  'token': 2816,
  'token_str': 'schools',
  'sequence': 'some people have schools that are diseased by viruses.'},
 {'score': 0.031352803111076355,
  'token': 8870,
  'token_str': 'cats',
  'sequence': 'some people have cats that are diseased by viruses.'}]

In [17]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased-finetuned-quantifier')
unmasker("[MASK] people have lungs which require good care.")

[{'score': 0.32520872354507446,
  'token': 2122,
  'token_str': 'these',
  'sequence': 'these people have lungs which require good care.'},
 {'score': 0.179448202252388,
  'token': 2035,
  'token_str': 'all',
  'sequence': 'all people have lungs which require good care.'},
 {'score': 0.17357008159160614,
  'token': 2070,
  'token_str': 'some',
  'sequence': 'some people have lungs which require good care.'},
 {'score': 0.06642021983861923,
  'token': 2087,
  'token_str': 'most',
  'sequence': 'most people have lungs which require good care.'},
 {'score': 0.05995144322514534,
  'token': 2116,
  'token_str': 'many',
  'sequence': 'many people have lungs which require good care.'}]

### finetuned distill-bert on new SI dataset

In [13]:
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased-finetuned-quantifier"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.save_pretrained('distilbert-base-uncased-finetuned-quantifier')

distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'


In [48]:
new_qdf['ft_distillbert_cw_prob'] = ''
new_qdf['ft_distillbert_top1'] = ''
new_qdf['ft_distillbert_top1_prob'] = ''
new_qdf['ft_distillbert_accuracy'] = 0

unmasker = pipeline('fill-mask', model='distilbert-base-uncased-finetuned-quantifier')

for i in new_qdf.index:
    seq = "[MASK] " + new_qdf['post_cw'][i]
    temp = unmasker(seq)
    new_qdf['ft_distillbert_top1'][i] = temp[0]['token_str']
    new_qdf['ft_distillbert_top1_prob'][i] = temp[0]['score']

    for item in temp:
        if item['token_str'] == new_qdf['cw'][i]:
            new_qdf['ft_distillbert_cw_prob'][i] = item['score']
            new_qdf['ft_distillbert_accuracy'][i] = 1

/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_5787/1315934054.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_qdf['ft_distillbert_top1'][i] = temp[0]['token_str']
/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_5787/1315934054.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_qdf['ft_distillbert_top1_prob'][i] = temp[0]['score']
/var/folders/xl/6x4fjht11dq2blkl91wqmnkh0000gn/T/ipykernel_5787/1315934054.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [49]:
new_qdf.head()

,cw,post_cw,condition,prediction,sentence,distillbert_cw_prob,distillbert_top1,distillbert_top1_prob,distillbert_cw_accuracy,ft_distillbert_cw_prob,ft_distillbert_top1,ft_distillbert_top1_prob,ft_distillbert_accuracy
0,some,people have lungs that are diseased by viruses.,baseline,so-so,Some people have lungs that are diseased by vi...,0.209061,some,0.209061,1,0.191318,these,0.295243,1
1,some,people have pets that are diseased by viruses.,baseline,good,Some people have pets that are diseased by vir...,0.289194,some,0.289194,1,0.197293,these,0.343162,1
2,all,people have lungs which require good care.,experimental,bad,Some people have lungs which require good care.,,some,0.146916,0,0.179448,these,0.325209,1
3,some,people have pets which require good care.,experimental,good,Some people have pets which require good care.,0.228662,some,0.228662,1,0.218351,these,0.292617,1
4,some,houses have rooms that are filled with kid's t...,baseline,so-so,Some houses have rooms that are filled with ki...,0.037414,these,0.082235,1,0.090154,these,0.553979,1


In [50]:
new_qdf.to_csv('/Users/yancong/Desktop/5 zili research/cogsci2022/data/SI_cw_quantifier.csv')

## presupposition

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
use_auth_token=True

In [8]:
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased-finetuned-existence"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.save_pretrained('distilbert-base-uncased-finetuned-existence')

distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'>>> DistilBERT number of parameters: {round(distilbert_num_parameters)}M'")
print(f"'>>> BERT number of parameters: 110M'")

'>>> DistilBERT number of parameters: 67M'
'>>> BERT number of parameters: 110M'


In [9]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased-finetuned-existence')
unmasker("[MASK] people have lungs which require good care.")

[{'score': 0.5311275720596313,
  'token': 2122,
  'token_str': 'these',
  'sequence': 'these people have lungs which require good care.'},
 {'score': 0.15924863517284393,
  'token': 2116,
  'token_str': 'many',
  'sequence': 'many people have lungs which require good care.'},
 {'score': 0.05643483251333237,
  'token': 2119,
  'token_str': 'both',
  'sequence': 'both people have lungs which require good care.'},
 {'score': 0.049704015254974365,
  'token': 2216,
  'token_str': 'those',
  'sequence': 'those people have lungs which require good care.'},
 {'score': 0.04197145253419876,
  'token': 2087,
  'token_str': 'most',
  'sequence': 'most people have lungs which require good care.'}]

In [10]:
df['ft_distillbert_cw_prob'] = ''
df['ft_distillbert_top1'] = ''
df['ft_distillbert_top1_prob'] = ''
df['ft_distillbert_accuracy'] = 0

unmasker = pipeline('fill-mask', model='distilbert-base-uncased-finetuned-existence')

for i in df.index:
    seq = df['pre_cw'][i] + " [MASK] " + df['post_cw'][i] + ' ' + df['post_cw_trigger'][i]
    temp = unmasker(seq)
    df['ft_distillbert_top1'][i] = temp[0]['token_str']
    df['ft_distillbert_top1_prob'][i] = temp[0]['score']

    for item in temp:
        if item['token_str'] == df['cw'][i]:
            df['ft_distillbert_cw_prob'][i] = item['score']
            df['ft_distillbert_accuracy'][i] = 1 # this is wrong, should be if bad, then 0; if good, then 1

<ipython-input-10-8c4cfe13be4e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ft_distillbert_top1'][i] = temp[0]['token_str']
<ipython-input-10-8c4cfe13be4e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ft_distillbert_top1_prob'][i] = temp[0]['score']
<ipython-input-10-8c4cfe13be4e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ft_distillbert_cw_prob'][i] = item['score']
<ipython-input-10-8c4cfe13be4e>:17: Sett

In [11]:
df.head()

,pre_cw,cw,post_cw,post_cw_trigger,condition,prediction,distillbert_cw_prob,distillbert_top1,distillbert_top1_prob,distillbert_cw_accuracy,ft_distillbert_cw_prob,ft_distillbert_top1,ft_distillbert_top1_prob,ft_distillbert_accuracy
0,Seth went to,jail,on Saturday night.,The guard spoke to him there for a while .,experimental,good,0.038628,sleep,0.115822,1,,school,0.082037,0
1,Seth went to a,restaurant,on Saturday night.,The guard spoke to him there for a while .,baseline,bad,0.064623,motel,0.137708,1,0.168983,restaurant,0.168983,1
2,Kristen went to a,restaurant,in the morning.,The waiter served her there quickly .,experimental,good,0.405458,restaurant,0.405458,1,0.57059,restaurant,0.57059,1
3,Kristen went to,jail,in the morning.,The waiter served her there quickly .,baseline,bad,,work,0.29974,0,,work,0.507694,0
4,Sue went to a,salon,in the fall.,The hairdresser addressed her there politely .,experimental,good,0.203876,salon,0.203876,1,,restaurant,0.177635,0


In [12]:
df.to_csv('/Users/yancong/Desktop/5 zili research/naacl2022_cong_shortPaper/data analysis/Presp_cw_bert.csv')